In [96]:
# EDA e Visualização de Dados
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from mlxtend.frequent_patterns import apriori, association_rules



# Carregar os Dados

In [97]:
# Carregar os dados
df_transacoes = pd.read_csv('datasets/transactions_by_dept.csv', sep=',')

In [98]:
df_transacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4539 entries, 0 to 4538
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   POS Txn  4539 non-null   uint64
 1   Dept     4539 non-null   object
 2   ID       4539 non-null   int64 
 3   Sales U  4539 non-null   int64 
dtypes: int64(2), object(1), uint64(1)
memory usage: 142.0+ KB


In [99]:
df_transacoes.head(10)

,POS Txn,Dept,ID,Sales U
0,16120100160021008773,0261:HOSIERY,250,2
1,16120100160021008773,0634:VITAMINS & HLTH AIDS,102,1
2,16120100160021008773,0879:PET SUPPLIES,158,2
3,16120100160021008773,0973:CANDY,175,2
4,16120100160021008773,0982:SPIRITS,176,1
5,16120100160021008773,0983:WINE,177,4
6,16120100160021008773,0991:TOBACCO,179,2
7,16120100160021008774,0597:HEALTH AIDS,93,1
8,16120100160021008774,0604:PERSONAL CARE,100,5
9,16120100160021008775,0819:PRE-RECORDED A/V,135,1


In [100]:
df_transacoes.tail(10)

,POS Txn,Dept,ID,Sales U
4529,16120100160162000841,0857:PC PERIPH/OFFICE ACC,155,1
4530,16120100160162000841,0931:BATH,165,1
4531,16120100160162000841,0941:BEDDING,167,1
4532,16120100160162000841,0991:TOBACCO,179,1
4533,16120100160162000842,0815:AUDIO ELECTRONICS,132,1
4534,16120100160162000843,0857:PC PERIPH/OFFICE ACC,155,1
4535,16120100160162000845,0395:MENS ATHLETIC SHOES,49,1
4536,16120100160162000845,0815:AUDIO ELECTRONICS,132,1
4537,16120100160162000846,0532:AMERICAN GREETINGS,72,1
4538,16120100160221001467,0066:VENDING/AMUSEMENT MA,242,1


# EDA

In [101]:
# Renomear as colunas para facilitar o entendimento
df_transacoes.rename(columns={'POS Txn': 'ID_Transacao', 'Dept': 'Departamento',
                              'ID': 'ID_Departamento', 'Sales U': 'Qtde_Vendida'}, inplace=True)

In [102]:
df_transacoes.head(10)

,ID_Transacao,Departamento,ID_Departamento,Qtde_Vendida
0,16120100160021008773,0261:HOSIERY,250,2
1,16120100160021008773,0634:VITAMINS & HLTH AIDS,102,1
2,16120100160021008773,0879:PET SUPPLIES,158,2
3,16120100160021008773,0973:CANDY,175,2
4,16120100160021008773,0982:SPIRITS,176,1
5,16120100160021008773,0983:WINE,177,4
6,16120100160021008773,0991:TOBACCO,179,2
7,16120100160021008774,0597:HEALTH AIDS,93,1
8,16120100160021008774,0604:PERSONAL CARE,100,5
9,16120100160021008775,0819:PRE-RECORDED A/V,135,1


In [103]:
# Quantidade de departamentos
df_transacoes['Departamento'].nunique()

160

In [104]:
# Quantidade de ID departamentos
df_transacoes['ID_Departamento'].nunique()

160

In [105]:
# Quantidade de Transacoes
df_transacoes['ID_Transacao'].nunique()

2064

In [106]:
# Um departamento aparece mais de 1x em uma transação?
len(df_transacoes.groupby(['ID_Transacao', 'Departamento'])) != len(df_transacoes)

False

In [107]:
# Checar se há quantidades negativas e 0
len(df_transacoes[df_transacoes['Qtde_Vendida'] <= 0])

149

In [108]:
# Remover as linhas com quantidade menor ou igual a 0
df_transacoes = df_transacoes[df_transacoes['Qtde_Vendida'] > 0]

In [109]:
# Quantidade de departamentos, ID departamentos, Transacoes
df_transacoes['Departamento'].nunique(), df_transacoes['ID_Departamento'].nunique(), df_transacoes['ID_Transacao'].nunique()

(156, 156, 1943)

In [110]:
# Quais departamentos tem maior volume de transações e maior quantidade de unidades vendidas?
# Parte 1: Volume de transacoes

# Quantidade de transcaoes unicas onde cada departamento aparece
contagem_transacoes_por_departamento = df_transacoes.value_counts('Departamento')

# Plot
fig_volume_transacoes = px.bar(contagem_transacoes_por_departamento, color=contagem_transacoes_por_departamento, orientation='h')
fig_volume_transacoes.update_layout(showlegend=False)
fig_volume_transacoes.show()

In [111]:
# Quais top 10 departamentos tem maior volume de transações e maior quantidade de unidades vendidas?
# Parte 1: Volume de transacoes

# Quantidade de transcaoes unicas onde cada departamento aparece
contagem_transacoes_por_departamento_top10 = df_transacoes.value_counts('Departamento').head(10)

# Plot
fig_volume_transacoes_top10 = px.bar(contagem_transacoes_por_departamento_top10, color=contagem_transacoes_por_departamento_top10, orientation='h')
fig_volume_transacoes_top10.update_layout(showlegend=False)
fig_volume_transacoes_top10.show()

In [112]:
# Qual departamentos tem maior volume de transações e maior quantidade de unidades vendidas?
# Parte 2: Quantidade de unidades vendidas

# Soma de Unidades Vendidas das transações onde cada departamento aparece
soma_quantidades_por_departamentos = df_transacoes.groupby('Departamento')['Qtde_Vendida'].sum().sort_values(ascending=False)

# Plot
fig_unidades_vendidas = px.bar(soma_quantidades_por_departamentos, color=soma_quantidades_por_departamentos, orientation='h')
fig_unidades_vendidas.update_layout(showlegend=False)
fig_unidades_vendidas.show()

In [113]:
# Qual o Top 10 departamentos tem maior volume de transações e maior quantidade de unidades vendidas?

# Parte 2: Quantidade de unidades vendidas
# Soma de Unidades Vendidas das transações onde cada departamento aparece
soma_quantidades_por_departamentos_top10 = df_transacoes.groupby('Departamento')['Qtde_Vendida'].sum().sort_values(ascending=False).head(10)

# Plot
fig_unidades_vendidas_top10 = px.bar(soma_quantidades_por_departamentos_top10, color=soma_quantidades_por_departamentos_top10, orientation='h')
fig_unidades_vendidas_top10.update_layout(showlegend=False)
fig_unidades_vendidas_top10.show()

In [114]:
# Unificar dois gráficos (Top 10), para mostrar lado a lado

# Criar subplots
fig_pergunta1 = make_subplots(rows=1, cols=2, subplot_titles=("Volume de Transações", "Unidades Vendidas"))

# Adicionar os gráficos aos subplots
for trace in fig_volume_transacoes_top10['data']:
    fig_pergunta1.add_trace(trace, row=1, col=1)

for trace in fig_unidades_vendidas_top10['data']:
    fig_pergunta1.add_trace(trace, row=1, col=2)

fig_pergunta1.update_layout(height=800, width=1000, title_text="Top 10 Departamentos", showlegend=False)
fig_pergunta1.show()


In [115]:
# Quais departamentos tem a maior variação nas unidades vendidas por transação? Top 20

# Calcular o desvio padrão das quantidades vendidas, agrupando por departamento

desvio_padrao_quantidades_por_departamentos_top20 = df_transacoes.groupby('Departamento')['Qtde_Vendida'].std().sort_values(ascending=False).head(20)

# Plot
fig_variacao_unidades_vendidas_top20 = px.bar(desvio_padrao_quantidades_por_departamentos_top20, color=desvio_padrao_quantidades_por_departamentos_top20, orientation='h')
fig_variacao_unidades_vendidas_top20.update_layout(showlegend=False)
fig_variacao_unidades_vendidas_top20.show()

In [116]:
# Qual é a distribuição de unidades vendidas por departamento nas diferentes transações?
# Avaliar as medidas de posição da variável 'Qtde_Vendida'
px.box(df_transacoes, x='Departamento', y='Qtde_Vendida')

In [117]:
# Quais departamentos tem a maior méidas nas unidades vendidas por transação? Top 20

# Calcular o desvio padrão das quantidades vendidas, agrupando por departamento

media_quantidades_por_departamentos_top20 = df_transacoes.groupby('Departamento')['Qtde_Vendida'].mean().sort_values(ascending=False).head(20)

# Plot
fig_media_unidades_vendidas_top20 = px.bar(media_quantidades_por_departamentos_top20, color=media_quantidades_por_departamentos_top20, orientation='h')
fig_media_unidades_vendidas_top20.update_layout(showlegend=False)
fig_media_unidades_vendidas_top20.show()

In [118]:
# Unificar dois gráficos (Top 20), para mostrar lado a lado

# Criar subplots
fig_pergunta4 = make_subplots(rows=1, cols=2, subplot_titles=("Desvio padrão", "Media"))

# Adicionar os gráficos aos subplots
for trace in fig_variacao_unidades_vendidas_top20['data']:
    fig_pergunta4.add_trace(trace, row=1, col=1)

for trace in fig_media_unidades_vendidas_top20['data']:
    fig_pergunta4.add_trace(trace, row=1, col=2)

fig_pergunta4.update_layout(height=800, width=1000, title_text="Top 20 Departamentos - Unidades Vendidas", showlegend=False)
fig_pergunta4.show()


# Preparação da Base Com Pivot Table

| ID_Trans | Prod1 | Prod2 | Prod3 | Prod4 |
|----------|-------|-------|-------|-------|
| 00000001 |   X   |       |   X   |       |
| 00000002 |   X   |   X   |   X   |       |
| 00000003 |       |       |   X   |   X   |


In [119]:
# Criar uma Pivot Table no Pandas com ID_Transacao como linha e Departamento como coluna, Soma de quantidade Vendida como valor

df_transacoes_pivot = df_transacoes.pivot_table(index='ID_Transacao', columns='Departamento', values='Qtde_Vendida', aggfunc='sum')

In [120]:
# Verificar estrutura Pivot Table
df_transacoes_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1943 entries, 16120100160021008773 to 16120100160221001467
Columns: 156 entries, 0002:BARBER RETAIL to 0999:UNSCANNED ITEMS
dtypes: float64(156)
memory usage: 2.3 MB


In [121]:
df_transacoes_pivot.head(20)

Departamento,0002:BARBER RETAIL,0009:FLORAL SERV/SUPPLIES,0012:LOCAL GIFTS/FOODS,0014:REPAIRS,0016:FINANCIAL SERVICES,0018:OTHER DIRECT SERVICE,0033:VENDING RETAIL,0056:SOUVENIR,0066:VENDING/AMUSEMENT MA,0071:BEAUTY RETAIL,...,0941:BEDDING,0961:GENERAL GROCERIES,0962:BEVERAGES,0965:PERISHABLES,0973:CANDY,0982:SPIRITS,0983:WINE,0984:BEER,0991:TOBACCO,0999:UNSCANNED ITEMS
ID_Transacao,,,,,,,,,,,,,,,,,,,,,
16120100160021008773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,1.0,4.0,NaN,2.0,NaN
16120100160021008774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16120100160021008775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
16120100160021008776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16120100160021008777,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
16120100160021008778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,1.0,NaN
16120100160021008779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,NaN,NaN
16120100160021008780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16120100160021008781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,NaN,2.0,NaN,NaN,NaN


In [122]:
# Converter os dados da Pivot table para Boolean (True/False)
df_transacoes_pivot = df_transacoes_pivot.map(lambda x: x > 0)

In [123]:
df_transacoes_pivot.head(20)

Departamento,0002:BARBER RETAIL,0009:FLORAL SERV/SUPPLIES,0012:LOCAL GIFTS/FOODS,0014:REPAIRS,0016:FINANCIAL SERVICES,0018:OTHER DIRECT SERVICE,0033:VENDING RETAIL,0056:SOUVENIR,0066:VENDING/AMUSEMENT MA,0071:BEAUTY RETAIL,...,0941:BEDDING,0961:GENERAL GROCERIES,0962:BEVERAGES,0965:PERISHABLES,0973:CANDY,0982:SPIRITS,0983:WINE,0984:BEER,0991:TOBACCO,0999:UNSCANNED ITEMS
ID_Transacao,,,,,,,,,,,,,,,,,,,,,
16120100160021008773,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,False,True,False
16120100160021008774,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16120100160021008775,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
16120100160021008776,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
16120100160021008777,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
16120100160021008778,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
16120100160021008779,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,True,False,False
16120100160021008780,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16120100160021008781,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,True,False,False,False


In [124]:
# Remover o título Departamento
df_transacoes_pivot = df_transacoes_pivot.rename_axis(None, axis=1)

In [125]:
# Dropar/Remover a Coluna ID_Transacao
df_transacoes_pivot.reset_index(inplace=True)

In [126]:
df_transacoes_pivot.drop(columns=['ID_Transacao'], inplace=True)

In [127]:
df_transacoes_pivot.head(20)

,0002:BARBER RETAIL,0009:FLORAL SERV/SUPPLIES,0012:LOCAL GIFTS/FOODS,0014:REPAIRS,0016:FINANCIAL SERVICES,0018:OTHER DIRECT SERVICE,0033:VENDING RETAIL,0056:SOUVENIR,0066:VENDING/AMUSEMENT MA,0071:BEAUTY RETAIL,...,0941:BEDDING,0961:GENERAL GROCERIES,0962:BEVERAGES,0965:PERISHABLES,0973:CANDY,0982:SPIRITS,0983:WINE,0984:BEER,0991:TOBACCO,0999:UNSCANNED ITEMS
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,True,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,True,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
